# Main parameters

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rxavier/poniard/blob/master/examples/02._main_parameters.ipynb)

This notebook outlines the most relevant options for Poniard estimators.

If you don't have it installed, please install from PyPI.

In [1]:
# %pip install poniard

At the core of Poniard lie the choice of estimators, metrics and CV strategy. While defaults might work for most cases, we try to keep it flexible.

## `estimators`

Estimators can be passed as a dict of `estimator_name: estimator_instance` or as a list of `estimator_instance`. In the latter, names will be obtained directly from the class.

Using a dictionary allows passing multiple instances of the same estimator with different hyperparameters.

In [2]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from poniard import PoniardClassifier

X, y = make_classification(n_classes=3, n_informative=3)
pnd = PoniardClassifier(estimators={"lr": LogisticRegression(max_iter=5000),
                                    "lr_no_penalty": LogisticRegression(max_iter=5000, penalty="none"),
                                    "lda": LinearDiscriminantAnalysis()})
pnd.setup(X, y)
pnd.fit()

Target info
-----------
Type: multiclass
Shape: (100,)
Unique values: 3

Main metric
-----------
roc_auc_ovr

Thresholds
----------
Minimum unique values to consider a feature numeric: 10
Minimum unique values to consider a categorical high cardinality: 20

Inferred feature types
----------------------


,numeric,categorical_high,categorical_low,datetime
0,0.0,,,
1,1.0,,,
2,2.0,,,
3,3.0,,,
4,4.0,,,
5,5.0,,,
6,6.0,,,
7,7.0,,,
8,8.0,,,
9,9.0,,,


Completed: 100%|██████████| 4/4 [00:00<00:00, 34.10it/s]


PoniardClassifier(estimators={'lr': LogisticRegression(max_iter=5000, random_state=0), 'lr_no_penalty': LogisticRegression(max_iter=5000, penalty='none', random_state=0), 'lda': LinearDiscriminantAnalysis()}, metrics=None,
    preprocess=True, scaler=standard, numeric_imputer=simple,
    custom_preprocessor=None, numeric_threshold=0.1,
    cardinality_threshold=20, cv=None, verbose=0,
    random_state=0, n_jobs=None, plugins=None,
    plot_options=PoniardPlotFactory())
            

Since we are in scikit-learn-land, most of the stuff you expect to work still works. For example, multilabel classification.

Here we had to use a dictionary because `estimator.__class__.__name__`, which is used for assigning a name to each estimator when a list is passed, would be the same for both `MultiOutputClassifier` and they would be overwritten.

In [3]:
from sklearn.datasets import make_multilabel_classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

X, y = make_multilabel_classification()
pnd = PoniardClassifier(estimators={"rf": OneVsRestClassifier(RandomForestClassifier()),
                                    "nb": OneVsRestClassifier(LogisticRegression())})
pnd.setup(X, y)
pnd.fit()

Target info
-----------
Type: multilabel-indicator
Shape: (100, 5)
Unique values: 2

Main metric
-----------
roc_auc

Thresholds
----------
Minimum unique values to consider a feature numeric: 10
Minimum unique values to consider a categorical high cardinality: 20

Inferred feature types
----------------------


,numeric,categorical_high,categorical_low,datetime
0,4.0,,0.0,
1,15.0,,1.0,
2,19.0,,2.0,
3,,,3.0,
4,,,5.0,
5,,,6.0,
6,,,7.0,
7,,,8.0,
8,,,9.0,
9,,,10.0,


/Users/rafxavier/Documents/Repos/personal/poniard/poniard/estimators/core.py:259: UserWarning: TargetEncoder is not supported for multilabel or multioutput targets. Switching to OrdinalEncoder.
  self.preprocessor = self._build_preprocessor()


Completed: 100%|██████████| 3/3 [00:01<00:00,  1.75it/s]      


PoniardClassifier(estimators={'rf': OneVsRestClassifier(estimator=RandomForestClassifier()), 'nb': OneVsRestClassifier(estimator=LogisticRegression())}, metrics=None,
    preprocess=True, scaler=standard, numeric_imputer=simple,
    custom_preprocessor=None, numeric_threshold=0.1,
    cardinality_threshold=20, cv=None, verbose=0,
    random_state=0, n_jobs=None, plugins=None,
    plot_options=PoniardPlotFactory())
            

In [4]:
pnd.get_results()

,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_precision_macro,train_precision_macro,test_recall_macro,train_recall_macro,test_f1_macro,train_f1_macro,fit_time,score_time
nb,0.707481,1.0,0.23,0.9850,0.586204,0.9992,0.539555,0.990167,0.535407,0.994497,0.015126,0.005446
rf,0.686371,1.0,0.23,1.0000,0.421012,1.0000,0.458030,1.000000,0.427833,1.000000,0.227035,0.035170
DummyClassifier,0.500000,0.5,0.09,0.0925,0.242000,0.2525,0.440000,0.440000,0.307975,0.320502,0.003301,0.005069


As you may have noticed, a dummy estimator is always included even if not explicitly set during initialization.

## `metrics`

Metrics can be passed as a list of strings, following the familiar scikit-learn nomenclature, or as a dict of `str: callable`. For convenience, it can also be a single string.

This restriction is in place to facilitate naming columns in the `get_results()` method.

In [5]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from poniard import PoniardRegressor

X, y = make_regression()
pnd = PoniardRegressor(metrics=["neg_median_absolute_error", "explained_variance"],
                       estimators=[LinearRegression()])
pnd.setup(X, y)
pnd.fit()

Target info
-----------
Type: continuous
Shape: (100,)
Unique values: 100

Main metric
-----------
neg_median_absolute_error

Thresholds
----------
Minimum unique values to consider a feature numeric: 10
Minimum unique values to consider a categorical high cardinality: 20

Inferred feature types
----------------------


,numeric,categorical_high,categorical_low,datetime
0,0.0,,,
1,1.0,,,
2,2.0,,,
3,3.0,,,
4,4.0,,,
5,5.0,,,
6,6.0,,,
7,7.0,,,
8,8.0,,,
9,9.0,,,


Completed: 100%|██████████| 2/2 [00:00<00:00, 34.40it/s]


PoniardRegressor(estimators=[LinearRegression()], metrics=['neg_median_absolute_error', 'explained_variance'],
    preprocess=True, scaler=standard, numeric_imputer=simple,
    custom_preprocessor=None, numeric_threshold=0.1,
    cardinality_threshold=20, cv=None, verbose=0,
    random_state=0, n_jobs=None, plugins=None,
    plot_options=PoniardPlotFactory())
            

In [6]:
pnd.get_results()

,test_neg_median_absolute_error,train_neg_median_absolute_error,test_explained_variance,train_explained_variance,fit_time,score_time
LinearRegression,-43.348074,-2.881251e-13,9.353761e-01,1.000000e+00,0.004579,0.001022
DummyRegressor,-132.406938,-1.253682e+02,-8.881784e-17,2.220446e-17,0.001443,0.000520


In [7]:
from sklearn.metrics import r2_score, make_scorer

def scaled_r2(y_true, y_pred):
    return round(r2_score(y_true, y_pred) * 100, 1)

pnd = PoniardRegressor(metrics={"scaled_r2": make_scorer(scaled_r2, greater_is_better=True),
                                "usual_r2": make_scorer(r2_score, greater_is_better=True)},
                       estimators=[LinearRegression()])
pnd.setup(X, y).fit().get_results()

Target info
-----------
Type: continuous
Shape: (100,)
Unique values: 100

Main metric
-----------
scaled_r2

Thresholds
----------
Minimum unique values to consider a feature numeric: 10
Minimum unique values to consider a categorical high cardinality: 20

Inferred feature types
----------------------


,numeric,categorical_high,categorical_low,datetime
0,0.0,,,
1,1.0,,,
2,2.0,,,
3,3.0,,,
4,4.0,,,
5,5.0,,,
6,6.0,,,
7,7.0,,,
8,8.0,,,
9,9.0,,,


Completed: 100%|██████████| 2/2 [00:00<00:00, 69.35it/s]


,test_scaled_r2,train_scaled_r2,test_usual_r2,train_usual_r2,fit_time,score_time
LinearRegression,92.90,100.0,0.928859,1.0,0.002379,0.000412
DummyRegressor,-4.06,0.0,-0.040490,0.0,0.000999,0.000319


## `cv`

Cross validation can be anything that scikit-learn accepts. By default, classification tasks will be paired with a `StratifiedKFold` if the target is binary, and `KFold` otherwise. Regression tasks use `KFold` by default.

`cv=int` or `cv=None` are internally converted to one of the above classes so that Poniard's `random_state` parameter can be passed on.

In [8]:
from IPython.utils import io
from sklearn.model_selection import RepeatedKFold

with io.capture_output() as c:
    pnd_5 = PoniardRegressor(cv=4).setup(X, y)
    pnd_none = PoniardRegressor(cv=None).setup(X, y)
    pnd_k = PoniardRegressor(cv=RepeatedKFold(n_splits=3)).setup(X, y)

In [9]:
print(pnd_5.cv, pnd_none.cv, pnd_k.cv, sep="\n")

KFold(n_splits=4, random_state=0, shuffle=True)
KFold(n_splits=5, random_state=0, shuffle=True)
RepeatedKFold(n_repeats=10, n_splits=3, random_state=0)


Note that even though we didn't specify `random_state` for the third estimator, it gets injected during setup.